# Assignment 3
## Numerical Optimization & Large Scale Linear Algebra

### Theofanis Nitsos, p3352325

In [1]:
import numpy as np
from scipy import sparse
from numpy.linalg import norm
import time
np.seterr(divide='ignore') # set to avoid interruptions or warnings in case division by zero errors might occur


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Load Data

The input file contains in compressed form the connectivity matrix for the webpages of Stanford University. Our goal is to extract the required information to create the directed graph. Thus which nodes point to which nodes; that is stored in the from_pages, to_pages arrays.

To avoid potential memory problems in case the transition matrix P cannot be stored in the main memory, we use the decomposition proposed in the paper $ P = D^{-1} G $  as well as the sparse library. 
Accordingly the adjacency matrix G, as well as the the the inverse of the diagonal matrix D holding outdegrees of the nodes are calculated

In [2]:
from_pages = []
to_pages = []
dict_out = {}
dict_in = {}

data = "stanweb.dat"

with open(data, 'r') as f:
    for line in f.readlines():
        from_page = int(line.split()[0]) - 1
        to_page = int(line.split()[1]) - 1
        from_pages.append(from_page)
        to_pages.append(to_page)
       
        if from_page not in dict_out.keys():
            dict_out[from_page] = 1
        else:
            dict_out[from_page] += 1
        if to_page not in dict_in.keys():
            dict_in[to_page] = [from_page]
        else:
            dict_in[to_page].append(from_page)

    # compute number of edges
    edges = len(from_pages)
    # compute number of nodes
    nodes = len(set(from_pages) | set(to_pages))
    for node in range(nodes):  
        if node not in dict_in.keys():
            dict_in[node] = []

    D = sparse.lil_matrix((nodes, 1))
    for key in dict_out.keys():
        D[key] = 1/dict_out[key]
    G = sparse.csr_matrix(([1]*edges,(to_pages, from_pages)), shape=(nodes, nodes))


## Question a

### i. Power method

The algorithm iteratively updates the rank vector r until the change (error) between consecutive rank vectors is less than the tolerance tol.
In each iteration:
- A new rank vector r_new is computed by multiplying the graph matrix G with the rank vector r, adjusted by the out-degree ratios out_a.
- The sum L of the new rank vector is calculated to ensure it sums to 1.
- r_new is adjusted to account for the total probability distribution.
- The error is updated as the Euclidean norm of the difference between r and r_new.
- On the second iteration, nodes that have already converged (change less than tol) are recorded in list_conv.
- The rank vector r is updated to r_new.

In [3]:
import time
import numpy as np

# Start timing
start_time = time.time()

# Constants
a = 0.85
tol = 1e-8

# Initialization
n = G.shape[0]
out_a = G.sum(axis=0).T / a
r = np.ones((n, 1)) / n
error = 1
counter = 0


# Pagerank iteration
while error > tol:
    counter += 1
    r_new = G.dot(r / out_a)
    L = r_new.sum()
    r_new += (1 - L) / n
    error = np.linalg.norm(r - r_new)
    if counter == 10:
        # list to return the nodes that converged from the 10th iteration
        list_conv_pm_10 = []
        # Iterate over all nodes
        for i in range(r.shape[0]):
            # Check if the node has converged
            if np.linalg.norm(r[i]-r_new[i]) < tol:
                # Add the node to the list if it has converged
                list_conv_pm_10.append(i)
    r = r_new
    
# Output results
print("Pagerank Algorithm requires: ", time.time() - start_time, "seconds to converge")
print("Number of iterations:", counter)

final_pagerank_pm = np.asarray(r).ravel()

Pagerank Algorithm requires:  1.6756408214569092 seconds to converge
Number of iterations: 72


### ii. Gauss Seidel 
- Use the Gauss-Seidel method to iteratively update the PageRank vector $x$ until the relative error falls below the tolerance level.
- Print the iteration count and relative error at each step.

In [4]:
import time
import numpy as np

# Start measuring time
start_t = time.time()

# Parameters
a = 0.85
tolerance = 1e-8

# Initial PageRank vector
pagerank_vector_old = np.ones(G.shape[0]) / G.shape[0]
iteration_count = 0
relative_error = 1
base_value = 1 / G.shape[0]

# Adjust the D matrix
adjusted_D = a * D
pagerank_vector = pagerank_vector_old.copy()

while relative_error > tolerance:
    iteration_count += 1
    for node_index in range(G.shape[0]):
        sum_predecessors = 0
        sum_successors = 0
        
        for neighbor in dict_in[node_index]:
            if neighbor < node_index:
                sum_predecessors += adjusted_D.data[neighbor][0] * pagerank_vector[neighbor]
                
            if neighbor > node_index:
                sum_successors += adjusted_D.data[neighbor][0] * pagerank_vector_old[neighbor]
                
                
        pagerank_vector[node_index] = base_value + sum_predecessors + sum_successors
        
    relative_error = np.linalg.norm(pagerank_vector - pagerank_vector_old) / np.linalg.norm(pagerank_vector)

    if iteration_count == 50:
        converged_nodes = []
        for node_index in range(pagerank_vector.shape[0]):
            if np.linalg.norm(pagerank_vector[node_index] - pagerank_vector_old[node_index]) < tolerance:
                converged_nodes.append(node_index)

    print('Iteration:', iteration_count, '=> Error:', relative_error)
    pagerank_vector_old = pagerank_vector.copy()

print("Execution time for PageRank with Gauss-Seidel:", time.time() - start_t, "seconds")
final_pagerank_gs = np.asarray(pagerank_vector).ravel()


Iteration: 1 => Error: 0.9976058648041445
Iteration: 2 => Error: 0.40541098260364294
Iteration: 3 => Error: 0.22047373339942714
Iteration: 4 => Error: 0.14008157663541068
Iteration: 5 => Error: 0.09165799424697829
Iteration: 6 => Error: 0.0620818940228253
Iteration: 7 => Error: 0.042895800711085216
Iteration: 8 => Error: 0.030071225622694424
Iteration: 9 => Error: 0.021304647568764656
Iteration: 10 => Error: 0.015210011046643018
Iteration: 11 => Error: 0.010921248126795863
Iteration: 12 => Error: 0.007874909962757243
Iteration: 13 => Error: 0.005696540133271812
Iteration: 14 => Error: 0.004130751624363217
Iteration: 15 => Error: 0.0030009128678329987
Iteration: 16 => Error: 0.00218324709419791
Iteration: 17 => Error: 0.0015901745892284232
Iteration: 18 => Error: 0.0011592462668741086
Iteration: 19 => Error: 0.0008457138252580716
Iteration: 20 => Error: 0.0006173470016753903
Iteration: 21 => Error: 0.00045087195460355347
Iteration: 22 => Error: 0.00032942821835479745
Iteration: 23 => Er

In [5]:
indices_pm = final_pagerank_pm.argsort()[-len(final_pagerank_pm):][::-1]
indices_gs = final_pagerank_gs.argsort()[-len(final_pagerank_gs):][::-1]
print("The are ", np.sum(indices_pm[:] != indices_gs[:]),"differences between the power method and Gauss Seidel")


The are  121568 differences between the power method and Gauss Seidel


In [6]:
# Number of top and bottom positions to compare
k = 50

# Compare the top k positions
top_k_indices_pm = indices_pm[:k]
top_k_indices_gs = indices_gs[:k]
top_differences = np.sum(top_k_indices_pm != top_k_indices_gs)

# Compare the bottom k positions
bottom_k_indices_pm = indices_pm[-k:]
bottom_k_indices_gs = indices_gs[-k:]
bottom_differences = np.sum(bottom_k_indices_pm != bottom_k_indices_gs)

# Print the results
print(f"There are {top_differences} differences in the top {k} positions.")
print(f"There are {bottom_differences} differences in the bottom {k} positions.")

There are 9 differences in the top 50 positions.
There are 0 differences in the bottom 50 positions.


### Comments - Question a

The results are not the same for the 2 methods. There are 121568 differences in the rankings. Further comparing the top and bottom positions of the power methods and the Gauss-Seidel methods we detect most of the differences in the top positions rather than the bottom. It is worth noting though that the top 10 positions are identical.

In terms of speed the Power method is a lot faster than the the Gauss-Seidel.

## Question 1b

Repeating the same as above with different parameters yields the following results

### i) Power Method

In [7]:
import time
import numpy as np

# Start timing
start_time = time.time()

# Constants
a = 0.99
tol = 1e-8

# Initialization
n = G.shape[0]
out_a = G.sum(axis=0).T / a
r = np.ones((n, 1)) / n
error = 1
counter = 0

# Pagerank iteration
while error > tol:
    counter += 1
    r_new = G.dot(r / out_a)
    L = r_new.sum()
    r_new += (1 - L) / n
    error = np.linalg.norm(r - r_new)
    
    # Track convergence on the 50th iteration
    if counter == 50:
        list_conv_50_pm = [i for i in range(n) if np.linalg.norm(r[i] - r_new[i]) < tol]
    
    r = r_new

# Output results
print("Time of Pagerank with Google's Algorithm:", time.time() - start_time, "seconds")
print("Number of iterations:", counter)

final_pagerank_pm_99 = np.asarray(r).ravel()

Time of Pagerank with Google's Algorithm: 9.224653244018555 seconds
Number of iterations: 1141


### ii) Gauss-Seidel

In [8]:
# Start measuring time
start_t = time.time()

# Parameters
a = 0.99
tolerance = 1e-8

# Initial PageRank vector
pagerank_vector_old = np.ones(G.shape[0]) / G.shape[0]
iteration_count = 0
relative_error = 1
base_value = 1 / G.shape[0]

# Adjust the D matrix
adjusted_D = a * D
pagerank_vector = pagerank_vector_old.copy()

while relative_error > tolerance:
    iteration_count += 1
    for node_index in range(G.shape[0]):
        sum_predecessors = 0
        sum_successors = 0
        
        for neighbor in dict_in[node_index]:
            if neighbor < node_index:
                sum_predecessors += adjusted_D.data[neighbor][0] * pagerank_vector[neighbor]
                
            if neighbor > node_index:
                sum_successors += adjusted_D.data[neighbor][0] * pagerank_vector_old[neighbor]
                
                
        pagerank_vector[node_index] = base_value + sum_predecessors + sum_successors
        
    relative_error = np.linalg.norm(pagerank_vector - pagerank_vector_old) / np.linalg.norm(pagerank_vector)
    

    print('Iteration:', iteration_count, '=> Error:', relative_error)
    pagerank_vector_old = pagerank_vector.copy()

print("Execution time for PageRank with Gauss-Seidel:", time.time() - start_t, "seconds")
final_pagerank_gs_99 = np.asarray(pagerank_vector).ravel()

Iteration: 1 => Error: 0.9981678259340901
Iteration: 2 => Error: 0.47185982372500446
Iteration: 3 => Error: 0.3100832127531674
Iteration: 4 => Error: 0.2352813309996406
Iteration: 5 => Error: 0.1862568586504489
Iteration: 6 => Error: 0.15353070497236124
Iteration: 7 => Error: 0.13004778661653357
Iteration: 8 => Error: 0.11252370771417988
Iteration: 9 => Error: 0.09899702839914701
Iteration: 10 => Error: 0.08826115114281581
Iteration: 11 => Error: 0.07954383560946526
Iteration: 12 => Error: 0.07232434911424068
Iteration: 13 => Error: 0.06624868022545356
Iteration: 14 => Error: 0.06106316840062764
Iteration: 15 => Error: 0.05658458227347894
Iteration: 16 => Error: 0.05267618438552159
Iteration: 17 => Error: 0.04923512934243682
Iteration: 18 => Error: 0.04618138419004752
Iteration: 19 => Error: 0.043452968689649166
Iteration: 20 => Error: 0.04100014479237045
Iteration: 21 => Error: 0.038783299195026544
Iteration: 22 => Error: 0.03676976102415316
Iteration: 23 => Error: 0.03493308827727398

In [9]:
indices_pm_99 = final_pagerank_pm_99.argsort()[-len(final_pagerank_pm_99):][::-1]
indices_gs_99 = final_pagerank_gs_99.argsort()[-len(final_pagerank_gs_99):][::-1]
print("The are ", np.sum(indices_pm_99[:] != indices_gs_99[:]),"differences")

The are  137868 differences


In [10]:
# Number of top and bottom positions to compare
k = 50

# Compare the top k positions
top_k_indices_pm = indices_pm[:k]
top_k_indices_gs = indices_gs[:k]
top_k_indices_pm_99 = indices_pm_99[:k]
top_k_indices_gs_99 = indices_gs_99[:k]
top_differences_99 = np.sum(top_k_indices_pm_99 != top_k_indices_gs_99)

# Compare the bottom k positions
bottom_k_indices_gs = indices_gs[-k:]
bottom_k_indices_pm = indices_pm[-k:]
bottom_k_indices_pm_99 = indices_pm_99[-k:]
bottom_k_indices_gs_99 = indices_gs_99[-k:]
bottom_differences_99 = np.sum(bottom_k_indices_pm_99 != bottom_k_indices_gs_99)

# Compare a = 0.85 with a = 0.99
top_differences_gs_a = np.sum(top_k_indices_gs != top_k_indices_gs_99)
top_differences_pm_a = np.sum(top_k_indices_pm != top_k_indices_pm_99)
bottom_differences_gs_a = np.sum(bottom_k_indices_gs != bottom_k_indices_gs_99)
bottom_differences_pm_a = np.sum(bottom_k_indices_pm != bottom_k_indices_pm_99)

# Print the results
print(f"There are {top_differences_99} differences in the top {k} positions between the 2 methods for alpha = 0.99.")
print(f"There are {bottom_differences_99} differences in the bottom {k} positions between the 2 methods for alpha = 0.99.")
print(f"There are {top_differences_pm_a} differences in the top {k} positions for the power method for the different alpha values.")
print(f"There are {top_differences_gs_a} differences in the top {k} positions for the Gauss-Seidel for the different alpha values.")
print(f"There are {bottom_differences_pm_a} differences in the bottom {k} positions for the power method for the different alpha values.")
print(f"There are {bottom_differences_gs_a} differences in the bottom {k} positions for the Gauss-Seidel for the different alpha values.")

There are 19 differences in the top 50 positions between the 2 methods for alpha = 0.99.
There are 49 differences in the bottom 50 positions between the 2 methods for alpha = 0.99.
There are 49 differences in the top 50 positions for the power method for the different alpha values.
There are 49 differences in the top 50 positions for the Gauss-Seidel for the different alpha values.
There are 49 differences in the bottom 50 positions for the power method for the different alpha values.
There are 49 differences in the bottom 50 positions for the Gauss-Seidel for the different alpha values.


In [11]:
print("Differences for a = 0.85 vs a = 0.99 can be found in ", sum(indices_pm[:50] != indices_pm_99[:50]), " out of the 50 first elements for the power method")


Differences for a = 0.85 vs a = 0.99 can be found in  49  out of the 50 first elements for the power method


In [12]:
print("Differences for a = 0.85 vs a = 0.99 can be found in ", sum(indices_gs[:50] != indices_gs_99[:50]), " out of the 50 first elements for the Gauss-Seidel")

Differences for a = 0.85 vs a = 0.99 can be found in  49  out of the 50 first elements for the Gauss-Seidel


### Comments - Question b

We observe that both methods converge significantly slower for alpha=0.99 which is to be expected. The Gauss Seidel method is again the slowest method. When compring the power method and the Gauss-Seidel for the different alpha values we notice: 
- From the top 50 elements the ranking of the first 49 nodes changed excluding the 1st node
- From the bottom 50 elements the ranking of the first 49 nodes changed excluding the last node

Further there are more differences between the ranking of the 2 methods for a = 0.99 than for a = 0.95 meaning the algorithms produce different results for different alpha values.

## ------------

### Question 1c

To figure out which nodes converge faster we ran the algorithms for power method and Gauss-Seidel again only this time we use a much lower tolerance $10^{-4}$ to drastically reduce iterations. As such we can compare the nodes that are in the same places between the power method and the Gauss Seidel method for only a few vs a lot of iterations. This should provide us with an indication of which nodes converge faster. 

It should be noted that our approach only checks whether the nodes are in the correct place. Another approach is finding the mean squared error between the nodes in the top and bottom.

In [13]:
# Power Method (same code as above with different tolerance)

# Start timing
start_time = time.time()

# Constants
a = 0.85
tol = 1e-4

# Initialization
n = G.shape[0]
out_a = G.sum(axis=0).T / a
r = np.ones((n, 1)) / n
error = 1
counter = 0


# Pagerank iteration
while error > tol:
    counter += 1
    r_new = G.dot(r / out_a)
    L = r_new.sum()
    r_new += (1 - L) / n
    error = np.linalg.norm(r - r_new)
    r = r_new
    
# Output results
print("Pagerank Algorithm requires: ", time.time() - start_time, "seconds to converge")
print("Number of iterations:", counter)

final_pagerank_pm_4 = np.asarray(r).ravel()

Pagerank Algorithm requires:  0.11764192581176758 seconds to converge
Number of iterations: 17


In [14]:
#indices_pm = final_pagerank_pm.argsort()[-len(final_pagerank_pm):][::-1]
indices_pm_4 = final_pagerank_pm_4.argsort()[-len(final_pagerank_pm_4):][::-1]
print("The are ", np.sum(indices_pm[:] != indices_pm_4[:]),"differences between the power method for 1e-8 and 1e-4 accuracy")

The are  279704 differences between the power method for 1e-8 and 1e-4 accuracy


In [15]:
# Number of top and bottom positions to compare
k = 100

# Compare the top k positions
top_k_indices_pm = indices_pm[:k]
top_k_indices_pm_4 = indices_pm_4[:k]
top_differences = np.sum(top_k_indices_pm != top_k_indices_pm_4)

# Compare the bottom k positions
bottom_k_indices_pm = indices_pm[-k:]
bottom_k_indices_pm_4 = indices_pm_4[-k:]
bottom_differences = np.sum(bottom_k_indices_pm != bottom_k_indices_pm_4)

# Print the results
print(f"There are {top_differences} differences in the top {k} positions.")
print(f"There are {bottom_differences} differences in the bottom {k} positions.")

There are 38 differences in the top 100 positions.
There are 99 differences in the bottom 100 positions.


In [16]:
# Gauss-Seidel method (same code as above with different tolerance)

# Start measuring time
start_t = time.time()

# Parameters
a = 0.85
tolerance = 1e-4

# Initial PageRank vector
pagerank_vector_old = np.ones(G.shape[0]) / G.shape[0]
iteration_count = 0
relative_error = 1
base_value = 1 / G.shape[0]

# Adjust the D matrix
adjusted_D = a * D
pagerank_vector = pagerank_vector_old.copy()

while relative_error > tolerance:
    iteration_count += 1
    for node_index in range(G.shape[0]):
        sum_predecessors = 0
        sum_successors = 0
        
        for neighbor in dict_in[node_index]:
            if neighbor < node_index:
                sum_predecessors += adjusted_D.data[neighbor][0] * pagerank_vector[neighbor]
                
            if neighbor > node_index:
                sum_successors += adjusted_D.data[neighbor][0] * pagerank_vector_old[neighbor]
                
                
        pagerank_vector[node_index] = base_value + sum_predecessors + sum_successors
        
    relative_error = np.linalg.norm(pagerank_vector - pagerank_vector_old) / np.linalg.norm(pagerank_vector)
    

    print('Iteration:', iteration_count, '=> Error:', relative_error)
    pagerank_vector_old = pagerank_vector.copy()

print("Execution time for PageRank with Gauss-Seidel:", time.time() - start_t, "seconds")
final_pagerank_gs_4 = np.asarray(pagerank_vector).ravel()

Iteration: 1 => Error: 0.9976058648041445
Iteration: 2 => Error: 0.40541098260364294
Iteration: 3 => Error: 0.22047373339942714
Iteration: 4 => Error: 0.14008157663541068
Iteration: 5 => Error: 0.09165799424697829
Iteration: 6 => Error: 0.0620818940228253
Iteration: 7 => Error: 0.042895800711085216
Iteration: 8 => Error: 0.030071225622694424
Iteration: 9 => Error: 0.021304647568764656
Iteration: 10 => Error: 0.015210011046643018
Iteration: 11 => Error: 0.010921248126795863
Iteration: 12 => Error: 0.007874909962757243
Iteration: 13 => Error: 0.005696540133271812
Iteration: 14 => Error: 0.004130751624363217
Iteration: 15 => Error: 0.0030009128678329987
Iteration: 16 => Error: 0.00218324709419791
Iteration: 17 => Error: 0.0015901745892284232
Iteration: 18 => Error: 0.0011592462668741086
Iteration: 19 => Error: 0.0008457138252580716
Iteration: 20 => Error: 0.0006173470016753903
Iteration: 21 => Error: 0.00045087195460355347
Iteration: 22 => Error: 0.00032942821835479745
Iteration: 23 => Er

In [17]:
indices_pm = final_pagerank_pm.argsort()[-len(final_pagerank_pm):][::-1]
indices_gs_4 = final_pagerank_gs_4.argsort()[-len(final_pagerank_gs_4):][::-1]
print("The are ", np.sum(indices_gs[:] != indices_gs_4[:]),"differences between the power method for 1e-8 and 1e-4 accuracy")

The are  242302 differences between the power method for 1e-8 and 1e-4 accuracy


In [18]:
# Number of top and bottom positions to compare
k = 100

# Compare the top k positions
top_k_indices_gs = indices_gs[:k]
top_k_indices_gs_4 = indices_gs_4[:k]
top_differences = np.sum(top_k_indices_gs != top_k_indices_gs_4)

# Compare the bottom k positions
bottom_k_indices_gs = indices_gs[-k:]
bottom_k_indices_gs_4 = indices_gs_4[-k:]
bottom_differences = np.sum(bottom_k_indices_gs != bottom_k_indices_gs_4)

# Print the results
print(f"There are {top_differences} differences in the top {k} positions.")
print(f"There are {bottom_differences} differences in the bottom {k} positions.")

There are 2 differences in the top 100 positions.
There are 99 differences in the bottom 100 positions.


### Comments - Question c

We would expect that more important nodes tend to converge faster. This is because important nodes usually have many incoming links from other high-ranking pages, which stabilizes their PageRank values faster. Non-important nodes on the other hand have fewer and less infuential incoming links leading to more fluctuations in their PageRank values during iteration.

The power method for a =0.85 and 72 iterations vs the power method with 17 iterations have:
- 38 differences in the top 100 nodes
- 99 differences in the bottom 100 nodes

The Gauss-Seidel method for a =0.85 and 56 iterations vs the power method with 26 iterations have:
- 2 differences in the top 100 nodes
- 99 difference in the bottom 100 nodes

Thus our insights are verified by these results. Top components of $\pi$ appear to converge faster, requiring less iterations compared to the less important nodes.

## Part 2

## Question a

We denote the transition matrix as $ G=a P+ \frac{1-a}{n}I $, where P is the matrix where each entry represents the probability of moving from one webpage to another.  
The Page rank vector is $\pi = (\pi_1, \pi_2, . . . , \pi_n)$  

The Page rank vector satisfied the equation $ \pi = G \pi \Rightarrow \pi = a P \pi + \frac{1-a}{n} \textbf{1} $

Adding the new webpage X the equations change as follows:   
$ G = \left(\matrix{G & \textbf{0}\\ \frac{1}{n+1}\textbf{1}^T & \frac{1}{n+1}  }\right) $, the new transition matrix with dimensions $ (n+1 \times n=1)$  

The new PageRank equations become:  
  
$ \hat{\pi} = a P \hat{\pi} + \frac{1-a}{n+1} \textbf{1} + \frac{a}{n+1} \textbf{1}x $  
  
$ x = \frac{1}{n+1} $    

Stationary equations

$ \left( \matrix{\hat{\pi} \\ x} \right) = \hat{G} \left( \matrix{\hat{\pi} \\ x} \right) $

Given $ x \approx \frac{1}{n+1} $, the new PageRank vector for the old pages $\hat{\pi} $ is slightly adjusted due to the normalization effect of adding a new page. The PageRanks of the old pages change marginally due to the added uniform probability $ \frac{1}{n+1} $

Since n is very large, the addition of a new page 𝑋 with no links does not significantly alter the PageRank of the existing pages. The PageRank values of the old pages are slightly reduced to accommodate the normalization of the new page's rank, but the overall distribution remains nearly the same.



Further in the simplified case of PageRank the PageRank value for any page u can be expressed as:

$ PR(u) = (1-a) + a * \sum{\frac{PR(u)}{L(u)}}$. Thus the pagerank value for a page u is dependent on the PageRank value for each page contained in the set containing all pages linking to page u, divided by the number $L(v)$ of links from page v. Since no page points to our website X then the changes will be minimal.

## Question b

Adding another webpage Y Y (with no in-links) that links to X will change the equations accordingly:

$ G = \left(\matrix{G & \textbf{0} & \textbf{0} \\ \frac{1}{n+2}\textbf{1}^T & 0 & 1 \\ \frac{1}{n+2}\textbf{1}^T & 1 & 0   }\right) $, the new transition matrix with dimensions $ (n+2 \times n + 2)$    

$ \left( \matrix{\hat{\pi} \\ x \\ y} \right) = G \left( \matrix{\hat{\pi} \\ x \\ y} \right) $  

Solving the above we can see that:

$ y = \frac{ (1-a) + ax }{n+2} $  
$ x = \frac{ (1-a) + ay }{n+2} $  

where x is the PageRank score of page X and y is the PageRank score of page Y

Adding a new page Y that links to X does indeed improve the PageRank of X. This is because X now has an incoming link, which contributes to its PageRank value. The PageRank of both 𝑋 and Y will be elevated slightly compared to having no in-links or out-links due to their mutual support in the link structure. Since though no other webpages will link to Y (especially webpages with high PageRank scores) it will have only a slight impact on the PageRank score of X.

## Question c

$ G = \left(\matrix{G & \textbf{0} & \textbf{0} & \textbf{0} \\ \frac{1}{n+3}\textbf{1}^T & 0 & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{n+3}\textbf{1}^T & 1 & 0 & 1 \\ \frac{1}{n+3}\textbf{1}^T & 1 & 1 & 0  }\right) $, the new transition matrix with dimensions $ (n+2 \times n + 2)$    

$ x = \frac{a}{2} y + \frac{a}{2} z +  \frac{1-a}{n+3} $  
$ y = \frac{a}{n + 3} \sum_{i=1}^n \pi' + \frac{a}{2} x + \frac{a}{2} z + \frac{1-a}{n+3} $  
$  z = \frac{a}{n + 3} \sum_{i=1}^n \pi' + \frac{a}{2} x + \frac{a}{2} z + \frac{1-a}{n+3} $

To maximize the PageRank of page X, we can strategically set up links between the newly created pages X, Y, and Z. The goal is to maximize the number of incoming links to X and to create a structure that ensures the PageRank "circulates" effectively among the new pages, boosting
X's rank. The links added in detail should be:
- Page Z links to X, additional incoming link to X
- Page Y links to X
- Page X links to Y and Z to ensure X contribution is divided and some portion circulates back through Y and Z to return to X
- Page Y links to Z Flow of pagerank from Y to Z, which will return to X
- Page Z links to Y Closes a loop ensuring PageRank keeps circulating


By creating a closed loop where each of the new pages X, Y, and Z links to each other, and ensuring that X has multiple incoming links, we maximize the PageRank of X. This setup ensures that PageRank circulates effectively among the new pages, concentrating the flow towards X. Again, the absence of links from websites with high PageRank scores towards our webpages X,Y and Z is crucial especially considering a very large n; thus the changes of the PageRank scores are not expected to be important.

## Question d

Adding links from your new page X (or Y or Z) to older, popular pages can influence the PageRank of X. 

In the PageRank algorithm, the rank of a page is determined by both the incoming links and how it distributes its rank to outgoing links. 
- Outgoing Links Distribute PageRank: When a page links to other pages, it distributes its PageRank among those pages. If X has many outgoing links, the rank is divided among them, potentially reducing the amount of PageRank retained.
- Linking to Popular Pages: Linking to popular pages doesn't directly increase the PageRank of X. Instead, it distributes X's rank to those popular pages, which can decrease the rank retained by X.

**Adding Links from X to Popular Pages**  
When X links to older, popular pages, the PageRank of X is likely to decrease. This is because:
- X will distribute its PageRank to these popular pages.
- Given that popular pages already have high PageRank, the contribution from X will not significantly boost X's own PageRank.

**Adding Links from Y or Z to Popular Pages**  
If you add links from Y or Z to older, popular pages, the effect is similar but might affect X differently:
- Y and Z will distribute their PageRank to the popular pages.
- The PageRank distributed by Y and Z will be less available to flow back to X through the links you set up among X, Y, and Z.

Adding links from X, Y, or Z to older, popular pages is not likely to improve the PageRank of X. Instead, it will distribute the PageRank of these pages to the popular pages, potentially lowering the overall PageRank of X.


## Question e

To further raise the PageRank of page X, we can leverage the concept of link farms effectively. Here are some steps and strategies based on the previous parts:
Steps to Raise PageRank of X
- Create a Cluster of Supporting Pages: Create several supporting pages (e.g., Y, Z, etc.) that will primarily link to X.
- Interlink Supporting Pages: Ensure that these supporting pages link to each other, creating a tightly-knit cluster that helps circulate the PageRank within the group before it gets directed to X.
- Maximize Incoming Links to X: Each supporting page should link to X. This maximizes the number of incoming links to X, boosting its PageRank.
- Limit Outgoing Links from X: Minimize the number of outgoing links from X to retain as much PageRank as possible. Only link to the most necessary pages.
- Distribute PageRank Evenly Among Supporting Pages: Ensure that the supporting pages do not have too many outgoing links to the original web pages. This keeps the PageRank circulating within the link farm before it is funneled to X.
Structure for a Link Farm with mmm Nodes  
    
To optimize the PageRank of X using a link farm with mmm supporting pages, we can design the structure as follows:
- Link Farm Nodes: $X,Y_1,Y_2,…,Y_m$
- Link Setup:
    - Each $ Y_i (for \; i=1,2,…,m) $ links to X.
    - Each $ Y_i$ links to $Y_{i+1}$ (creating a circular structure among supporting pages).
    - Optionally, some links from $Y_i$ to $Yi+k $ to create redundancy in the link farm, ensuring robust circulation of PageRank.
- Minimize Links from X: X should ideally have minimal or no outgoing links. If necessary, it should link to a few highly relevant pages.


## Question 2e

Based on the previous questions it is obvious that increasing the PageRank of a webpage is not straightforward. The PageRank of a webpage can be increased if more and more pages point to that to webpage. Of course, if those webpages have low PageRank then their contribution will be of little significance. On the contrary if webpages with higher PageRank point to our wepbage then its score will increase. Thus it is difficult to "cheat" the system and increase the pagerank of X further. Nevertheless, in general, the more pages pointing to a webpage, the higher the PageRank of that page. Moreover, interconnections amongst those pages could also be helpful.